In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393319 sha256=ae96376efe53d6d625bb4123b373a5c59ac0199d1397f8b8e67cc34088e6117e
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import os, re, csv, math, codecs
import fasttext as ft
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [44]:
# Construct a tf.data.Dataset
data = pd.read_csv('/content/drive/MyDrive/processed_sentiment_140_en_mr.csv',encoding='utf-8', names = ['polarity','id','text','lang'])

In [45]:
data= data.dropna().reset_index(drop=True)

In [ ]:
data.head()

In [47]:
#processing label of training/testing data
label_data=data['polarity'].values
senti=[0,4]
mapping = {}
for x in range(len(senti)):
    mapping[senti[x]] = x

# integer representation
for x in range(len(label_data)):
    label_data[x] = mapping[label_data[x]]

#conveting to one-hot-encoding
y_data = to_categorical(label_data)

In [ ]:
y_data[0]

In [50]:
# Text cleaning function
def text_cleaner(line):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u2066"
                               u"\u2069"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #remove punctuation and special characters
    #line= re.sub(r'[^\w\s]','',line)
    punc = '''؛₹|!‼→¿()-[]{};–৷।:،۔•„'“‘—’”"…\,<>=./?@#$%^&*_~»«'''
    line=re.sub(r'http\S+', '', line)
    line=emoji_pattern.sub(r'', line)
    line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
    for ele in line:
        if ele in punc:
            line = line.replace(ele, "")
            line=re.sub(r"^\s+|\s+$", "", line)
            line = re.sub(' +', ' ', line) # removing extra white spaceS
            #line= clean(line, no_emoji=True)
    return line

In [51]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer

In [52]:
data_x=data['text'].tolist()

In [53]:
raw_docs_train, raw_docs_test, y_train, y_test = train_test_split(data_x, y_data,  test_size=0.2, random_state=4)

In [ ]:
#pre_processing train/test data
MAX_NB_WORDS = 100000
# tokenizer = RegexpTokenizer(r'\w+')
tokenizer = WhitespaceTokenizer()

# raw_docs_train = train_df[1].tolist()
# raw_docs_test = test_df[1].tolist()
num_classes = 2

print("pre-processing train data...")
processed_docs_train = []
for line in tqdm(raw_docs_train):
    line=text_cleaner(line)
    tokens = tokenizer.tokenize(line)
    #filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(tokens))
#end for
print("pre-processing test data...")
processed_docs_test = []
for line in tqdm(raw_docs_test):
    #line=text_cleaner(line)
    tokens = tokenizer.tokenize(line)
    #filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(tokens))
#end for

data['doc_len'] = data['text'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(data['doc_len'].mean() + data['doc_len'].std()).astype(int)

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))



In [55]:
#pad sequences
word_seq_train = keras.utils.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = keras.utils.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [ ]:
#loading embedding
import fasttext as ft
ft_model = ft.load_model("/content/drive/MyDrive/cc.mr.300.bin")

In [ ]:
#embedding matrix
embed_dim = 300
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = ft_model.get_word_vector(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [58]:
#model parameters
num_filters = 12
weight_decay = 1e-4

==============================cnn======================

In [ ]:
# CNN architecture
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_filters,  7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='softmax'))  #multi-label (k-hot encoding)
adam = tf.optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [60]:
#training params
batch_size = 256
num_epochs = 20

In [61]:
checkpoint_path = "/content/drive/MyDrive/Twitter_dataset/trainined_cnn_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                 verbose=1)

In [ ]:
#model training
hist = model.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs,validation_split=0.1, shuffle=True, verbose=1,callbacks=[cp_callback])

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Make predictions on the test data
y_pred = model.predict(word_seq_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoded y_test to categorical labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate precision
precision = precision_score(y_test_labels, y_pred)

# Calculate recall
recall = recall_score(y_test_labels, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("F1 Score:", f1)


fastext-cnn-LANGUAGE

==============================================LSTM====================================================

In [64]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

In [ ]:
#LSTM
lstm_out = 128
model_LSTM = keras.Sequential()
model_LSTM.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model_LSTM.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model_LSTM.add(Dense(128, activation = 'relu'))
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(64, activation = 'relu'))
model_LSTM.add(Dense(num_classes, activation='softmax'))  #multi-label (k-hot encoding)
adam = tf.optimizers.Adam()
model_LSTM.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_LSTM.summary()

In [66]:
#training params
batch_size_LSTM = 256
num_epochs_LSTM = 20

In [67]:
checkpoint_path = "training/LSTM/language_detection/trained_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#model training_LSTM
hist_LSTM = model_LSTM.fit(word_seq_train, y_train, batch_size=batch_size_LSTM, epochs=num_epochs_LSTM,validation_split=0.1, shuffle=True, verbose=1,callbacks=[cp_callback])

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Make predictions on the test data
y_pred = model_LSTM.predict(word_seq_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoded y_test to categorical labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate precision
precision = precision_score(y_test_labels, y_pred)

# Calculate recall
recall = recall_score(y_test_labels, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("F1 Score:", f1)


fastext-lstm-LANGUAGE

====================================
 GRU  ==========================

In [ ]:
from keras.layers import Dense, Embedding,  GRU, SpatialDropout1D, Bidirectional, Dropout

In [ ]:
gru_out = 128
model_GRU = keras.Sequential()
model_GRU.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model_GRU.add(Bidirectional(GRU(gru_out, dropout=0.2)))
model_GRU.add(Dense(128, activation='relu'))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(64, activation='relu'))
model_GRU.add(Dense(num_classes, activation='softmax'))  # multi-label (k-hot encoding)
adam = tf.optimizers.Adam()
model_GRU.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model_GRU.summary()

In [ ]:
#training params
batch_size = 256
num_epochs = 20

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Twitter_dataset/trainined_gru_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                 verbose=1)

In [ ]:
#model training
hist = model_GRU.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs,validation_split=0.1, shuffle=True, verbose=1,callbacks=[cp_callback])

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Make predictions on the test data
y_pred = model_GRU.predict(word_seq_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoded y_test to categorical labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate precision
precision = precision_score(y_test_labels, y_pred)

# Calculate recall
recall = recall_score(y_test_labels, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("F1 Score:", f1)


Muril-gru-LANGUAGE